# Airlift planning

You are in charge of determining which subset of the following requirements should be shipped on the next C-17 to another base:

| Requirement      | Capability Value | Weight (tons) | Volume ($\text{m}^3$) |
|:-----------------|:-----------------|:--------------|:-------------|
| Large munitions  | 50               | 43            | 250          |
| Small munitions  | 30               | 17            | 130          |
| Food             | 80               | 26            | 370          |
| Medical supplies | 40               | 4             | 180          |
| Repair parts     | 70               | 35            | 400          |

The C-17 has a weight capacity of 80 tons, and a volume capacity of 700 $\text{m}^3$. The goal is to maximize the total capability value of the requirements shipped.


* Formulate this problem as a dynamic program by giving its shortest/longest path representation.


* Solve this dynamic program. 
    - What is the maximum possible total capability value?
    - Which requirements should be put on the plane to achieve this maximum total capability value?

In [ ]:
# Write your code here


_Write your notes here. Double-click to edit._
